In [44]:
#Donald Dunagan and Andrew Stout
#14 December 2018
#CSCI 6560 Term Project
#Applications of Evolutionary Computation to Regression 
#with Multilayer Perceptron Artificial Neural Networks

# Utilizing the DEAP Package: Distributed Evolutionary Algorithms in Python

In [3]:
#    DEAP is free software: you can redistribute it and/or modify
#    it under the terms of the GNU Lesser General Public License as
#    published by the Free Software Foundation, either version 3 of
#    the License, or (at your option) any later version.
#
#    DEAP is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
#    GNU Lesser General Public License for more details.
#
#    You should have received a copy of the GNU Lesser General Public
#    License along with DEAP. If not, see <http://www.gnu.org/licenses/>.

# Set Up Environment

In [4]:
#imports

import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.utils import shuffle
from sklearn.metrics import make_scorer, mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, GridSearchCV
from scipy import stats
import random, operator, time, itertools, math, numpy
from deap import algorithms, base, creator, tools
import warnings
import array

In [5]:
#import Parkinson's Dataset and visualize

PDDF =  pd.read_csv("C:\\Users\\gray\\Desktop\\Evolutionary_Computing\\Term_Project\\Parkinson's_Data.csv")
print ('The shape of the dataframe is:',PDDF.shape)
PDDF.head()

The shape of the dataframe is: (5875, 22)


,subject#,age,sex,test_time,motor_UPDRS,total_UPDRS,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,...,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE
0,1,72,0,5.6431,28.199,34.398,0.00662,0.000034,0.00401,0.00317,...,0.230,0.01438,0.01309,0.01662,0.04314,0.014290,21.640,0.41888,0.54842,0.16006
1,1,72,0,12.6660,28.447,34.894,0.00300,0.000017,0.00132,0.00150,...,0.179,0.00994,0.01072,0.01689,0.02982,0.011112,27.183,0.43493,0.56477,0.10810
2,1,72,0,19.6810,28.695,35.389,0.00481,0.000025,0.00205,0.00208,...,0.181,0.00734,0.00844,0.01458,0.02202,0.020220,23.047,0.46222,0.54405,0.21014
3,1,72,0,25.6470,28.905,35.810,0.00528,0.000027,0.00191,0.00264,...,0.327,0.01106,0.01265,0.01963,0.03317,0.027837,24.445,0.48730,0.57794,0.33277
4,1,72,0,33.6420,29.187,36.375,0.00335,0.000020,0.00093,0.00130,...,0.176,0.00679,0.00929,0.01819,0.02036,0.011625,26.126,0.47188,0.56122,0.19361


#### Initial shuffling and splitting

In [6]:
#shuffle the data
PDDF = shuffle(PDDF)

#isolate features
features_to_drop = ['subject#','age','sex','test_time','motor_UPDRS','total_UPDRS']
just_features = PDDF.drop(features_to_drop,axis=1)
    
#isolate motor_UPDRS
just_motor = PDDF.motor_UPDRS
   
#isolate total_UPDRS
just_total = PDDF.total_UPDRS

#split into training and testing data
training_features = just_features[:4700]
testing_features = just_features[4700:]
    
training_motor = just_motor[:4700]
testing_motor = just_motor[4700:]
    
training_total = just_total[:4700]
testing_total = just_total[4700:] 

#### A function for shuffling and splitting the data

In [7]:
def shuffleSplit():
    '''
    Shuffles and splits the DataFrame 80%-20%
    '''
    global PDDF
    
    #shuffle the data
    PDDF = shuffle(PDDF)

    #isolate features
    features_to_drop = ['subject#','age','sex','test_time','motor_UPDRS','total_UPDRS']
    global just_features
    just_features = PDDF.drop(features_to_drop,axis=1)
    
    #isolate motor_UPDRS
    global just_motor
    just_motor = PDDF.motor_UPDRS
   
    #isolate total_UPDRS
    global just_total
    just_total = PDDF.total_UPDRS

    #split into training and testing data
    global training_features
    training_features = just_features[:4700]
    global testing_features
    testing_features = just_features[4700:]
    
    global training_motor
    training_motor = just_motor[:4700]
    global testing_motor
    testing_motor = just_motor[4700:]
    
    global training_total
    training_total = just_total[:4700]
    global testing_total
    testing_total = just_total[4700:]  
    
    return

In [8]:
#cross validation needs to return the mean absolute error (MAE) for each fold
#so i have to make a custom scoring object

#create scorer object
myScorer = make_scorer(mean_absolute_error)

In [9]:
PDDF.head()

,subject#,age,sex,test_time,motor_UPDRS,total_UPDRS,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,...,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE
1289,9,68,0,119.350,18.243,25.497,0.01019,0.000086,0.00588,0.00628,...,0.417,0.02039,0.02407,0.04128,0.06116,0.063458,18.202,0.61687,0.76106,0.417660
5445,40,85,1,43.713,16.441,26.961,0.00511,0.000031,0.00275,0.00293,...,0.276,0.01647,0.01998,0.02701,0.04942,0.032450,21.433,0.56208,0.60798,0.189550
1235,9,68,0,84.348,17.000,24.834,0.00894,0.000078,0.00422,0.00495,...,0.628,0.03816,0.03793,0.05232,0.11449,0.039944,18.459,0.78490,0.79372,0.385950
5837,42,61,0,86.767,25.598,36.464,0.00169,0.000013,0.00067,0.00085,...,0.118,0.00669,0.00758,0.01146,0.02008,0.006883,28.084,0.57206,0.54540,0.084809
2718,20,67,0,25.832,11.841,17.841,0.01016,0.000082,0.00576,0.00528,...,0.718,0.04605,0.04675,0.05161,0.13815,0.028634,19.248,0.53051,0.81041,0.246750


# Scikit Learn MLP

## Baseline for untuned, default MLP regressor

### Calculate testing MAE of an untuned MLP regressor targeting total_UPDRS

In [33]:
#train NN
MLP = MLPRegressor()
MLP.fit(training_features,training_total)
#make predictions using the testing data
predicts = (MLP.predict(testing_features)).astype(float)
#compare predicts and actual and record mean absolut error
MAE = mean_absolute_error(testing_total,predicts)
print("The testing total UPDRS MAE is: {}".format(MAE))

The testing total UPDRS MAE is: 8.882597232339744


### Idea is to use evolutionary computation to optimize MLP hyperparameters

### Fitness Evaluation Functon for Total UPDRS

In [11]:
'''
size_layer1, activation_function, optimization_algorithm, alpha value, size_layer2
'''
def MLP_Eval_Total(vals):
    
    #genotypes is real valued so some alleles need to be 
    #rounded to nearest int
    
    vals[0] = round(vals[0])
    vals[1] = round(vals[1])
    vals[2] = round(vals[2])
    vals[4] = round(vals[4])
    
    #if params are out of bounds, place them back in bounds
    if vals[0] <= 0:
        vals[0] = 1        #size of hidden layer1
    if vals[0] > 100:
        vals[0] = 100
        
    if vals[4] <= 0:
        vals[4] = 1        #size of hidden layer2
    if vals[4] > 100:
        vals[4] = 100 

    if vals[1] < 0:
        vals[1] = 0     #activation function
    if vals[1] > 3:
        vals[1] = 3
        
    if vals[2] < 0:
        vals[2] = 0   #optimization algorithm
    if vals[2] > 2:
        vals[2] = 2
        
    if vals[3] < 0.0001:
        vals[3] = .0001
    if vals[3] > 1000:
        vals[3] = 1000
        
    #assign vals to int alleles (if necessary)
    if vals[1] == 0:
        AF = 'identity'
    if vals[1] == 1:
        AF = 'logistic'    #activation function
    if vals[1] == 2:
        AF = 'tanh'
    if vals[1] == 3:
        AF = 'relu'

    if vals[2] == 0:
        OA = 'lbfgs'
    if vals[2] == 1:
        OA = 'sgd'    #optimization algorithm
    if vals[2] == 2:
        OA = 'adam'
    
    #construct phenotype
    MLP = MLPRegressor(hidden_layer_sizes=(vals[0],vals[4],), activation=AF, solver=OA, alpha = vals[3])

    #some phenotypes will throw an error
    try:
        #train NN
        MLP.fit(training_features, training_total)
        #make predictions using testing data
        predicts = (MLP.predict(training_features)).astype(float)
        #compare predicts and actual and record mean absolute error
        MAE = mean_absolute_error(training_total,predicts)
        goodPredicts = True
    #if there is an error in fitting or evaluating the MLP,
    #this candidate solution is clearly bad
    except:
        MAE = 1000
    
    #return MAE as fitness
    return (MAE,)

### Function for Generating Individuals

In [12]:
#generates an individual with random number of nodes in hidden layer 1,
#random activation function, random weight optimizer, random alpha value, and 
#random number of nodes in hidden layer 2
#within the restrictions
def Generate_Individual():
    individual = [random.uniform(1,101),random.uniform(0,4), random.uniform(0,3), random.uniform(0.001,1000),random.uniform(0,101)]
    return individual

### Create FitnessMin and Individual classes

In [13]:
#python throws a warning due to an intentional duplicate class
import warnings
warnings.filterwarnings('ignore')

In [14]:
#the fitness function is to be minimized
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
#an individual is composed of a list of vals with fitness to be minimized
creator.create("Individual", list, fitness=creator.FitnessMin)

### Toolbox

In [15]:
#instantiate toolbox
toolbox = base.Toolbox()
#tool to create an individual
toolbox.register("individual", tools.initIterate, creator.Individual, Generate_Individual)
#tool to create a population
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
#tool for fitness evaluation
toolbox.register("evaluate", MLP_Eval_Total)

#tool for crossover (one point)
toolbox.register("mate", tools.cxOnePoint)
#mutation is gaussian mutation
toolbox.register("mutate", tools.mutGaussian, mu=0,sigma=1,indpb=0.2)
#tool for tournament selection
toolbox.register("select", tools.selTournament, tournsize=3)

#### Run the algorithm 30 times and record best solutions

In [16]:
#list for best solutions from each run
HOF_list_hyperparams = []

#run the algorithm 30 times
for i in range(30):
    #generate the intial population (2 * number of dimensions)
    pop = toolbox.population(n=10)
    #Object for recording best candidate to ever live
    HOF = tools.HallOfFame(1)
    #run the algorithm
    result, log = algorithms.eaSimple(pop, toolbox,halloffame=HOF, cxpb=0.7, mutpb=1, ngen=10, verbose=False)
    print("Run number {} complete".format(i))
    HOF_list_hyperparams.append(HOF)

Run number 0 complete
Run number 1 complete
Run number 2 complete
Run number 3 complete
Run number 4 complete
Run number 5 complete
Run number 6 complete
Run number 7 complete
Run number 8 complete
Run number 9 complete
Run number 10 complete
Run number 11 complete
Run number 12 complete
Run number 13 complete
Run number 14 complete
Run number 15 complete
Run number 16 complete
Run number 17 complete
Run number 18 complete
Run number 19 complete
Run number 20 complete
Run number 21 complete
Run number 22 complete
Run number 23 complete
Run number 24 complete
Run number 25 complete
Run number 26 complete
Run number 27 complete
Run number 28 complete
Run number 29 complete


#### Calculate testing MAEs for best solutions

In [32]:
#list for MAEs
hyperparam_MAEs = []

for i in range(30):
    #genotypes is real valued so some alleles need to be 
    #rounded to nearest int

    vals = HOF_list_hyperparams[i].items[0]

    vals[0] = round(vals[0])
    vals[1] = round(vals[1])
    vals[2] = round(vals[2])
    vals[4] = round(vals[4])

    #if params are out of bounds, place them back in bounds
    if vals[0] <= 0:
        vals[0] = 1        #size of hidden layer1
    if vals[0] > 100:
        vals[0] = 100

    if vals[4] <= 0:
        vals[4] = 1        #size of hidden layer2
    if vals[4] > 100:
        vals[4] = 100 

    if vals[1] < 0:
        vals[1] = 0     #activation function
    if vals[1] > 3:
        vals[1] = 3

    if vals[2] < 0:
        vals[2] = 0   #optimization algorithm
    if vals[2] > 2:
        vals[2] = 2

    if vals[3] < 0.0001:
        vals[3] = .0001
    if vals[3] > 1000:
        vals[3] = 1000

    #assign vals to int alleles (if necessary)
    if vals[1] == 0:
        AF = 'identity'
    if vals[1] == 1:
        AF = 'logistic'    #activation function
    if vals[1] == 2:
        AF = 'tanh'
    if vals[1] == 3:
        AF = 'relu'

    if vals[2] == 0:
        OA = 'lbfgs'
    if vals[2] == 1:
        OA = 'sgd'    #optimization algorithm
    if vals[2] == 2:
        OA = 'adam'

    #construct phenotype
    MLP = MLPRegressor(hidden_layer_sizes=(vals[0],vals[4],), activation=AF, solver=OA, alpha = vals[3])

    #train NN
    MLP.fit(training_features, training_total)
    #make predictions using testing data
    predicts = (MLP.predict(testing_features)).astype(float)
    #compare predicts and actual and record mean absolute error
    MAE = mean_absolute_error(testing_total,predicts)
    hyperparam_MAEs.append(MAE)

#### Calculate mean testing MAE and std

In [37]:
hyperparam_accuracy = np.mean(hyperparam_MAEs)
hyperparam_std = np.std(hyperparam_MAEs)

print("Accuracy: {}".format(hyperparam_accuracy))
print("Standard deviation: {}".format(hyperparam_std))

Accuracy: 8.42584936244169
Standard deviation: 0.11287507607013775


# Evolving NN Weights

## 5-Fold Cross Validation to Determine Optimum Number of Hidden Nodes 

#### Compare 8 nodes in hidden layer to 128

In [10]:
#5-fold cross validated untuned MLP targeting total_UPDRS

MLP = MLPRegressor(hidden_layer_sizes=(8,))

#5-fold CV
vals = cross_val_score(MLP,training_features,training_total,cv=5, scoring=myScorer)
print("The 5-fold CV'd MAE with 8 nodes in the hidden layer is: {}".format(vals.mean()))

C:\Users\gray\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\gray\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\gray\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


The 5-fold CV'd MAE with 8 nodes in the hidden layer is: 8.672209545913429


C:\Users\gray\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


#### ^ the NN is not converging with only 8 hidden nodes. Needs more

In [177]:
#5-fold cross validated untuned MLP targeting total_UPDRS

MLP = MLPRegressor(hidden_layer_sizes=(128,))

#5-fold CV
vals = cross_val_score(MLP,training_features,training_total,cv=5, scoring=myScorer)
print("The 5-fold CV'd MAE with 128 nodes in the hidden layer is: {}".format(vals.mean()))

The 5-fold CV'd MAE with 128 nodes in the hidden layer is: 8.538145873205464


#### 128 converges and has better MAE, but can we get the same result with less nodes?

In [21]:
#5-fold cross validated untuned MLP targeting total_UPDRS

MLP = MLPRegressor(hidden_layer_sizes=(64,))

#5-fold CV
vals = cross_val_score(MLP,training_features,training_total,cv=5, scoring=myScorer)
print("The 5-fold CV'd MAE with 64 nodes in the hidden layer is: {}".format(vals.mean()))

The 5-fold CV'd MAE with 64 nodes in the hidden layer is: 8.510845443717818


#### Accuracy is the same with half the nodes. Let's try 32

In [22]:
#5-fold cross validated untuned MLP targeting total_UPDRS

MLP = MLPRegressor(hidden_layer_sizes=(32,))

#5-fold CV
vals = cross_val_score(MLP,training_features,training_total,cv=5, scoring=myScorer)
print("The 5-fold CV'd MAE with 32 nodes in the hidden layer is: {}".format(vals.mean()))

The 5-fold CV'd MAE with 32 nodes in the hidden layer is: 8.565301580052068


#### Accuracy is worse. 64 nodes in the hidden layer is our sweet spot

## Evolving NN Weights with a Genetic Algorithm

#### Evaluation Function

In [17]:
def evalNN(individual):
    """
    Takes the actual UPDRS values (either motor or total) and an np 
    array of length 1088, extracts the weight matrices,
    runs the neural network, and calculates the mean absolute 
    error between predicted UPDRS and actual UPDRS
    """
    #extract weight matrices
    weights1 = np.array(individual[0:1024]).reshape(16,64)
    weights2 = np.array(individual[1024:]).reshape(64,1)
    
    
    #perform matrix operations that take place inside the 
    #neural network
    
    #matrix multiplication between feature matrix and weights of hidden layer
    hidden = np.matmul(training_features,weights1)
    #add biases of hidden layer to this
    hidden = hidden + 1
    
    #perform ReLu activation function
    np.maximum(hidden,0,hidden)
    
    #matrix multiplication between the result of the hidden layer and 
    #the weights of the output layer
    final = np.matmul(hidden,weights2)

    #activation function for output node of regression is linear
    #so this is result of the neural network
    
    #calculate mean absolute error between predicted and actual and return 
    MAE =  mean_absolute_error(training_total,final)
    return (MAE,)

#### Function for generating an individual

In [18]:
def Generate_Individual():
    #individuals generated from the standard normal distribution: mu=0, std=1
    individual = np.random.normal(0,1,1088)
    return individual

#### Create FitnessMin and Individual classes

In [19]:
#the fitness function is to be minimized
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
#an individual is composed of a list of vals with fitness to be minimized
creator.create("Individual", list, fitness=creator.FitnessMin)

#### Toolbox

In [20]:
#instantiate toolbox
toolbox = base.Toolbox()
#tool to create an individual
toolbox.register("individual", tools.initIterate, creator.Individual, Generate_Individual)
#tool to create a population
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
#tool for fitness evaluation
toolbox.register("evaluate", evalNN)

#tool for crossover (one point)
toolbox.register("mate", tools.cxOnePoint)
#mutation is gaussian mutation
toolbox.register("mutate", tools.mutGaussian, mu=0,sigma=1.0,indpb=(1/1088))
#tool for tournament selection
toolbox.register("select", tools.selTournament, tournsize=50)

#### Run the algorithm 30 times and record best solutions

In [21]:
#list for best solutions from each run
HOF_list_GA = []

#run the algorithm 30 times
for i in range(30):
    #generate the intial population (0.5 * number of dimensions)
    pop = toolbox.population(n=544)
    #Object for recording best candidate to ever live
    HOF = tools.HallOfFame(1)
    #run the algorithm
    result, log = algorithms.eaSimple(pop, toolbox,halloffame=HOF, cxpb=0.7, mutpb=1, ngen=40, verbose=False)
    print("Run number {} complete".format(i))
    HOF_list_GA.append(HOF)

Run number 0 complete
Run number 1 complete
Run number 2 complete
Run number 3 complete
Run number 4 complete
Run number 5 complete
Run number 6 complete
Run number 7 complete
Run number 8 complete
Run number 9 complete
Run number 10 complete
Run number 11 complete
Run number 12 complete
Run number 13 complete
Run number 14 complete
Run number 15 complete
Run number 16 complete
Run number 17 complete
Run number 18 complete
Run number 19 complete
Run number 20 complete
Run number 21 complete
Run number 22 complete
Run number 23 complete
Run number 24 complete
Run number 25 complete
Run number 26 complete
Run number 27 complete
Run number 28 complete
Run number 29 complete


#### Calculate testing MAEs for best solutions

In [40]:
#list for MAEs
GA_MAEs = []

for i in range(30):
    MAE = evalNN(HOF_list_GA[i].items[0])[0]
    GA_MAEs.append(MAE)

#### Calculate mean testing MAE and std

In [41]:
GA_accuracy = np.mean(GA_MAEs)
GA_std = np.std(GA_MAEs)

print("Accuracy: {}".format(GA_accuracy))
print("Standard deviation: {}".format(GA_std))

Accuracy: 8.489249371294234
Standard deviation: 0.0558674384083221


## Evolve NN Weights with Evolution Strategies

### Creator

In [22]:
#fitness is to be minimized, an individual is an array of doubles, a strategy is an array of doubles
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", array.array, typecode="d", fitness=creator.FitnessMin, strategy=None)
creator.create("Strategy", array.array, typecode="d")

### Function for Generating Individuals

In [23]:
#generates object variables and strategy parameters for an individual
def generateES(icls, scls):
    ind = icls(random.uniform(-3, 3) for _ in range(1088))
    ind.strategy = scls(random.uniform(.001, 3) for _ in range(1088))
    return ind

### Function for Ensuring Sigmas are Valid

In [24]:
#if sigmas are below the threshold, push them back in bounds
def checkStrategy(minstrategy):
    def decorator(func):
        def wrappper(*args, **kargs):
            children = func(*args, **kargs)
            for child in children:
                for i, s in enumerate(child.strategy):
                    if s < minstrategy:
                        child.strategy[i] = minstrategy
            return children
        return wrappper
    return decorator

### Toolbox

In [25]:
toolbox = base.Toolbox()
toolbox.register("individual", generateES, creator.Individual, creator.Strategy)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
#two point crossover on both individuals and their strategies
toolbox.register("mate", tools.cxESTwoPoint)
toolbox.register("mutate", tools.mutESLogNormal, c=1.0, indpb=0.03)
toolbox.register("select", tools.selTournament, tournsize=50)
toolbox.register("evaluate", evalNN)

toolbox.decorate("mate", checkStrategy(.01))
toolbox.decorate("mutate", checkStrategy(.01))

### Run the Algorithm 30 times and recor best solutions

In [26]:
#list for best solutions from each run
HOF_list_ES = []

#run the algorithm 60 times
for i in range(60):
    #generate the initial population
    pop = toolbox.population(544)
    #object for recording best candidate to ever live
    HOF = tools.HallOfFame(1)
    #run the algorithm
    pop, logbook = algorithms.eaMuCommaLambda(pop, toolbox, mu=544, lambda_=544, 
        cxpb=0.6, mutpb=0.3, ngen=40, halloffame=HOF, verbose = False)
    print("Run number {} complete".format(i))
    HOF_list_ES.append(HOF)

Run number 0 complete
Run number 1 complete
Run number 2 complete
Run number 3 complete
Run number 4 complete
Run number 5 complete
Run number 6 complete
Run number 7 complete
Run number 8 complete
Run number 9 complete
Run number 10 complete
Run number 11 complete
Run number 12 complete
Run number 13 complete
Run number 14 complete
Run number 15 complete
Run number 16 complete
Run number 17 complete
Run number 18 complete
Run number 19 complete
Run number 20 complete
Run number 21 complete
Run number 22 complete
Run number 23 complete
Run number 24 complete
Run number 25 complete
Run number 26 complete
Run number 27 complete
Run number 28 complete
Run number 29 complete
Run number 30 complete
Run number 31 complete
Run number 32 complete
Run number 33 complete
Run number 34 complete
Run number 35 complete
Run number 36 complete
Run number 37 complete
Run number 38 complete
Run number 39 complete
Run number 40 complete
Run number 41 complete
Run number 42 complete
Run number 43 complet

#### Calculate testing MAEs for best solutions

In [42]:
#list for MAEs
ES_MAEs = []

for i in range(30):
    MAE = evalNN(HOF_list_ES[i].items[0])[0]
    ES_MAEs.append(MAE)

#### Calculate mean testing MAE and std

In [43]:
ES_accuracy = np.mean(ES_MAEs)
ES_std = np.std(ES_MAEs)

print("Accuracy: {}".format(ES_accuracy))
print("Standard deviation: {}".format(ES_std))

Accuracy: 8.415751939190853
Standard deviation: 0.05690728515311217
